In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from pysr import PySRRegressor

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
import scipy.io
from scipy.interpolate import griddata

In [3]:
import pandas as pd

In [4]:
data = scipy.io.loadmat(r'data\burgers.mat')

In [9]:
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Sat Aug 27 07:36:14 2016',
 '__version__': '1.0',
 '__globals__': [],
 't': array([[ 0. ],
        [ 0.1],
        [ 0.2],
        [ 0.3],
        [ 0.4],
        [ 0.5],
        [ 0.6],
        [ 0.7],
        [ 0.8],
        [ 0.9],
        [ 1. ],
        [ 1.1],
        [ 1.2],
        [ 1.3],
        [ 1.4],
        [ 1.5],
        [ 1.6],
        [ 1.7],
        [ 1.8],
        [ 1.9],
        [ 2. ],
        [ 2.1],
        [ 2.2],
        [ 2.3],
        [ 2.4],
        [ 2.5],
        [ 2.6],
        [ 2.7],
        [ 2.8],
        [ 2.9],
        [ 3. ],
        [ 3.1],
        [ 3.2],
        [ 3.3],
        [ 3.4],
        [ 3.5],
        [ 3.6],
        [ 3.7],
        [ 3.8],
        [ 3.9],
        [ 4. ],
        [ 4.1],
        [ 4.2],
        [ 4.3],
        [ 4.4],
        [ 4.5],
        [ 4.6],
        [ 4.7],
        [ 4.8],
        [ 4.9],
        [ 5. ],
        [ 5.1],
        [ 5.2],
        [

In [54]:
x_val = [-8.     -7.9375 -7.875  -7.8125 -7.75   -7.6875 -7.625  -7.5625 -7.5
 -7.4375 -7.375  -7.3125 -7.25   -7.1875 -7.125  -7.0625 -7.     -6.9375
 -6.875  -6.8125 -6.75   -6.6875 -6.625  -6.5625 -6.5    -6.4375 -6.375
 -6.3125 -6.25   -6.1875 -6.125  -6.0625 -6.     -5.9375 -5.875  -5.8125
 -5.75   -5.6875 -5.625  -5.5625 -5.5    -5.4375 -5.375  -5.3125 -5.25
 -5.1875 -5.125  -5.0625 -5.     -4.9375 -4.875  -4.8125 -4.75   -4.6875
 -4.625  -4.5625 -4.5    -4.4375 -4.375  -4.3125 -4.25   -4.1875 -4.125
 -4.0625 -4.     -3.9375 -3.875  -3.8125 -3.75   -3.6875 -3.625  -3.5625
 -3.5    -3.4375 -3.375  -3.3125 -3.25   -3.1875 -3.125  -3.0625 -3.
 -2.9375 -2.875  -2.8125 -2.75   -2.6875 -2.625  -2.5625 -2.5    -2.4375
 -2.375  -2.3125 -2.25   -2.1875 -2.125  -2.0625 -2.     -1.9375 -1.875
 -1.8125 -1.75   -1.6875 -1.625  -1.5625 -1.5    -1.4375 -1.375  -1.3125
 -1.25   -1.1875 -1.125  -1.0625 -1.     -0.9375 -0.875  -0.8125 -0.75
 -0.6875 -0.625  -0.5625 -0.5    -0.4375 -0.375  -0.3125 -0.25   -0.1875
 -0.125  -0.0625  0.      0.0625  0.125   0.1875  0.25    0.3125  0.375
  0.4375  0.5     0.5625  0.625   0.6875  0.75    0.8125  0.875   0.9375
  1.      1.0625  1.125   1.1875  1.25    1.3125  1.375   1.4375  1.5
  1.5625  1.625   1.6875  1.75    1.8125  1.875   1.9375  2.      2.0625
  2.125   2.1875  2.25    2.3125  2.375   2.4375  2.5     2.5625  2.625
  2.6875  2.75    2.8125  2.875   2.9375  3.      3.0625  3.125   3.1875
  3.25    3.3125  3.375   3.4375  3.5     3.5625  3.625   3.6875  3.75
  3.8125  3.875   3.9375  4.      4.0625  4.125   4.1875  4.25    4.3125
  4.375   4.4375  4.5     4.5625  4.625   4.6875  4.75    4.8125  4.875
  4.9375  5.      5.0625  5.125   5.1875  5.25    5.3125  5.375   5.4375
  5.5     5.5625  5.625   5.6875  5.75    5.8125  5.875   5.9375  6.
  6.0625  6.125   6.1875  6.25    6.3125  6.375   6.4375  6.5     6.5625
  6.625   6.6875  6.75    6.8125  6.875   6.9375  7.      7.0625  7.125
  7.1875  7.25    7.3125  7.375   7.4375  7.5     7.5625  7.625   7.6875
  7.75    7.8125  7.875   7.9375]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (905309179.py, line 1)

In [ ]:
[ 2.22044605e-16+0.00000000e+00j -4.31008024e-11+2.83319071e-13j
  4.77340778e-10-7.60066541e-11j  8.87246263e-10-3.11330925e-10j
  2.30088188e-09-1.49099002e-09j  2.05590329e-10-2.52376418e-10j
 -8.57071636e-10+1.74256676e-09j -1.72800524e-11-2.98105784e-10j
  1.55524801e-10+1.67176302e-09j  1.61544274e-10+3.99561898e-10j
 -1.15876361e-09-1.34407696e-09j  1.76102121e-09+1.16148822e-09j
  2.19709945e-11+4.90222902e-12j  6.03967709e-10+4.57853486e-11j
  2.34413697e-10-1.53987796e-11j -1.24891281e-09+1.00369483e-09j
  9.17191240e-10-8.23140688e-10j -9.21167309e-12+9.82570559e-10j
  9.93911980e-10-3.82260183e-09j  8.72612593e-10-5.38757766e-10j
  3.26395660e-09+2.88532627e-09j  1.12709853e-09-1.18532228e-09j
  6.29961078e-09+1.30563061e-09j  5.08642262e-09-3.87727345e-11j
  6.84934412e-09+1.13088777e-10j  9.99911978e-09+2.94583618e-10j
  1.50579150e-08+5.19849136e-11j  2.32268952e-08-4.32140829e-09j
  2.86515045e-08+8.23156187e-10j  3.88736388e-08-1.34243069e-09j
  5.21160224e-08-3.85793548e-10j  7.06323776e-08+1.36192630e-09j
  9.02085508e-08-3.01362802e-10j  1.17326183e-07-1.01571243e-10j
  1.49058516e-07+4.21255540e-10j  1.93701800e-07+2.04721433e-11j
  2.46220854e-07-1.33715561e-09j  3.06992205e-07+2.00294555e-10j
  3.82286229e-07-6.53844493e-10j  4.72154051e-07+1.62021345e-10j
  5.79662133e-07+9.45161644e-10j  7.04671838e-07-8.54616304e-10j
  8.54375781e-07+2.33562267e-11j  1.02440099e-06-3.30590577e-10j
  1.22916337e-06+1.03216524e-10j  1.46428507e-06-1.25755187e-09j
  1.72948165e-06+1.08051053e-09j  2.03678461e-06-6.86189606e-10j
  2.38519223e-06+4.72725461e-09j  2.78030205e-06+2.99291943e-10j
  3.22686808e-06+1.24800059e-09j  3.72740227e-06+3.73201918e-10j
  4.28356874e-06-6.77780381e-10j  4.91408012e-06-2.13415815e-10j
  5.60693314e-06-3.22585525e-11j  6.37871205e-06-3.15666240e-09j
  7.22426894e-06-5.77061427e-10j  8.15714113e-06+3.68989313e-10j
  9.18017388e-06-1.42534366e-10j  1.03011405e-05+3.07030987e-09j
  1.15175798e-05-4.31143551e-10j  1.28431201e-05-1.93188513e-10j
  1.42802302e-05+5.36135545e-11j  1.58363215e-05+1.34738127e-10j
  1.75152872e-05-9.57807291e-10j  1.93209537e-05-5.29857751e-10j
  2.12609179e-05-2.14195575e-09j  2.33402358e-05+5.46080903e-13j
  2.55650494e-05+9.10637722e-10j  2.79371127e-05-4.79233435e-10j
  3.04671573e-05-6.06594211e-12j  3.31558590e-05+2.28252079e-10j
  3.60105714e-05+5.85702738e-10j  3.90365160e-05-3.00111475e-10j
  4.22362327e-05+2.42901156e-10j  4.56158929e-05-4.50518690e-10j
  4.91810087e-05+1.99971642e-09j  5.29327765e-05+3.08776318e-10j
  5.68776274e-05-1.03631834e-10j  6.10214433e-05-6.79301058e-11j
  6.53611898e-05+1.14230929e-09j  6.99120297e-05-8.84572174e-11j
  7.46688269e-05-8.98523537e-10j  7.96368551e-05-1.76272432e-09j
  8.48201155e-05-5.77364242e-10j  9.02229878e-05+1.08447105e-09j
  9.58441098e-05-3.27001731e-10j  1.01695865e-04+2.76650402e-10j
  1.07765011e-04-7.62789721e-11j  1.14065124e-04+1.63940761e-09j
  1.20599927e-04+4.20765156e-10j  1.27366501e-04+4.96101242e-10j
  1.34367446e-04-1.93143624e-09j  1.41606580e-04+2.52765659e-10j
  1.49079529e-04-1.80103006e-09j  1.56798760e-04-4.63733826e-11j
  1.64759816e-04-1.19074940e-09j  1.72954871e-04+9.42123157e-10j
  1.81398207e-04-2.31123818e-10j  1.90083736e-04+1.63855813e-09j
  1.99013820e-04+2.01531609e-10j]

In [6]:
# Extract variables
t = data['t'].flatten()  # 1D array for time
x = data['x'].flatten()  # 1D array for space
usol = np.real(data['usol'])  # Take only the real part if `usol` is complex

In [7]:
# Create a meshgrid for all space-time combinations
T, X = np.meshgrid(t, x, indexing='ij')

In [8]:
t

array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,
        1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,
        2.2,  2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,
        4.4,  4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,
        5.5,  5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,
        6.6,  6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,
        7.7,  7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,
        8.8,  8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,
        9.9, 10. ])

In [9]:
T

array([[ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0.1,  0.1,  0.1, ...,  0.1,  0.1,  0.1],
       [ 0.2,  0.2,  0.2, ...,  0.2,  0.2,  0.2],
       ...,
       [ 9.8,  9.8,  9.8, ...,  9.8,  9.8,  9.8],
       [ 9.9,  9.9,  9.9, ...,  9.9,  9.9,  9.9],
       [10. , 10. , 10. , ..., 10. , 10. , 10. ]])

In [10]:
usol

array([[ 2.22044605e-16, -4.31008024e-11,  4.77340778e-10, ...,
         1.81398207e-04,  1.90083736e-04,  1.99013820e-04],
       [ 4.85722573e-16,  4.28848658e-11, -4.78508736e-10, ...,
         2.08914676e-04,  2.18644653e-04,  2.28635026e-04],
       [ 1.04083409e-15, -4.26225071e-11,  4.79589604e-10, ...,
         2.40188515e-04,  2.51064261e-04,  2.62217886e-04],
       ...,
       [-7.80625564e-17,  4.34146816e-11, -4.73120376e-10, ...,
         1.17517897e-04,  1.23619041e-04,  1.29915917e-04],
       [-1.82145965e-17, -4.33821173e-11,  4.74640424e-10, ...,
         1.36050551e-04,  1.42928147e-04,  1.50017788e-04],
       [ 2.08166817e-17,  4.32728853e-11, -4.76044357e-10, ...,
         1.57232607e-04,  1.64970064e-04,  1.72934908e-04]])

In [11]:
# Flatten the arrays to create a table
df = pd.DataFrame({
    't': T.flatten(),       # Time values
    'x': X.flatten(),       # Spatial values
    'u': usol.flatten()     # Solution values
})

In [14]:
# Save the dataframe for PyRegressor
df.to_csv("data/burgers_data.csv", index=False)

In [12]:
df

,t,x,u
0,0.0,-8.0000,2.220446e-16
1,0.0,-7.9375,-4.310080e-11
2,0.0,-7.8750,4.773408e-10
3,0.0,-7.8125,8.872463e-10
4,0.0,-7.7500,2.300882e-09
...,...,...,...
25851,10.0,7.6875,1.424345e-04
25852,10.0,7.7500,1.497234e-04
25853,10.0,7.8125,1.572326e-04
25854,10.0,7.8750,1.649701e-04


In order to discover PDEs in the data using symbolic regression, we need to compute spatial and temporal derivatives *u(x,t)*.
we can use either finite differences method or more accurately make use of Neural Networks to compute the derivatives.

**Using Finite Differences**

In [13]:
# Calculate the spatial and temporal step sizes
dx = x[1] - x[0]  # Assuming uniform spacing
dt = t[1] - t[0]  # Assuming uniform spacing

In [14]:
# First-order derivatives
u_x = np.gradient(usol, axis=1, edge_order=2) / dx  # du/dx
u_t = np.gradient(usol, axis=0, edge_order=2) / dt  # du/dt

In [15]:
# Second-order spatial derivatives
u_xx = np.gradient(u_x, axis=1, edge_order=2) / dx  # d²u/dx²

In [16]:
# Flatten the arrays and organize into a DataFrame
df_2 = pd.DataFrame({
    't': T.flatten(),
    'x': X.flatten(),
    'u': usol.flatten(),
    'u_t': u_t.flatten(),
    'u_x': u_x.flatten(),
    'u_xx': u_xx.flatten(),
})

In [21]:
# Save the DataFrame for use in symbolic regression tools
df_2.to_csv("data/burgers_data_with_derivatives.csv", index=False)

In [17]:
df_2

,t,x,u,u_t,u_x,u_xx
0,0.0,-8.0000,2.220446e-16,1.179612e-15,-5.197957e-09,1.874079e-07
1,0.0,-7.9375,-4.310080e-11,1.717322e-09,3.818724e-09,1.011259e-07
2,0.0,-7.8750,4.773408e-10,-1.912823e-08,7.442777e-09,8.615683e-08
3,0.0,-7.8125,8.872463e-10,-3.554737e-08,1.458833e-08,-1.031682e-07
4,0.0,-7.7500,2.300882e-09,-9.211231e-08,-5.453247e-09,-3.188157e-07
...,...,...,...,...,...,...
25851,10.0,7.6875,1.424345e-04,2.074320e-04,1.147614e-04,5.809782e-05
25852,10.0,7.7500,1.497234e-04,2.162733e-04,1.183847e-04,5.769673e-05
25853,10.0,7.8125,1.572326e-04,2.250676e-04,1.219735e-04,5.787001e-05
25854,10.0,7.8750,1.649701e-04,2.340832e-04,1.256184e-04,5.826516e-05


In [18]:
from itertools import combinations_with_replacement

In [19]:
# Define features and interactions
features = ['u', 'u_x', 'u_xx']
target = df_2['u_t']  # Target variable
feature_matrix = pd.DataFrame()

In [20]:
# Add raw features
for feature in features:
    feature_matrix[feature] = df_2[feature]

In [21]:
# Add interaction terms
for degree in range(2, 4):  # Include up to degree-3 interactions
    for combo in combinations_with_replacement(features, degree):
        name = '*'.join(combo)
        term = df_2[combo[0]]
        for var in combo[1:]:
            term *= df_2[var]
        feature_matrix[name] = term

In [22]:
# Save for PyRegressor
feature_matrix['u_t'] = target
feature_matrix.to_csv("data/prepared_data.csv", index=False)

In [10]:
# Load the prepared dataset
data_2 = pd.read_csv("data/prepared_data.csv")
Xf = data_2.drop(columns=['u_t']).values  # Features
u_t_pred = data_2['u_t'].values  # Target

In [36]:
# Train the symbolic regression model

model = PySRRegressor(
    niterations=1000,  # Number of iterations (adjust as necessary)
    
    # List of binary operators (two inputs)
    binary_operators=[
        "+",    # Addition
        "-"    # Subtraction 
    ],
    
    # List of unary operators (one input)
    unary_operators=[
        "sin",      # Sine function
        "cos"      # Cosine function
    ],
    
    # Define any custom operators (optional)
    extra_sympy_mappings={"inv": lambda x: 1/x},  # Example of a custom operator
    variable_names=[f"x{i}" for i in range(X.shape[1])],
    model_selection='accuracy',                    # Enable model selection mode
    population_size=1000,               # Population size for evolutionary search
    maxsize=20                          # Maximum complexity of equations
)
    





c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:922: FutureWarning: `variable_names` is a data-dependent parameter and should be passed when fit is called. Ignoring parameter; please pass `variable_names` during the call to fit instead.
  warnings.warn(


In [ ]:
model.fit(Xf, yf)

c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(
c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2059: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure 

Compiling Julia backend...


[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
[ Info: Started!



Expressions evaluated per second: 1.360e+05
Head worker occupation: 15.4%
Progress: 10 / 15000 total iterations (0.067%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.664e-02  1.594e+01  y = x₆
2           9.355e-03  5.757e-01  y = sin(-1.28e-05)
3           1.387e-03  1.908e+00  y = x₄ - x₁
4           1.370e-03  1.301e-02  y = sin(x₄) - x₁
5           1.359e-03  7.453e-03  y = sin(sin(x₄)) - x₁
6           1.353e-03  4.333e-03  y = sin(sin(sin(x₄))) - x₁
7           1.350e-03  2.377e-03  y = sin(sin(sin(sin(x₄)))) - x₁
8           1.345e-03  3.821e-03  y = sin(sin(sin(x₄ - -0.0028987))) - x₁
9           1.342e-03  2.284e-03  y = sin(sin(sin(sin(x₄) + 0.0028733))) - x₁
10          1.339e-03  2.517e-03  y = sin(sin(x₁) - (x₁ + (x₁ - sin(x₄))))
11          1.245e-03  7.243e-02  y = sin(sin(sin(sin(sin(sin(x₇ - x₆) - x₁)))))
12          1.195e-03  4.113e-02  y = si

In [38]:
# Load the prepared dataset
data_2 = pd.read_csv("data/prepared_data.csv")
Xf = data_2.drop(columns=['u_t']).values # Features
u_t_pred = data_2['u_t'].values  # Target

In [ ]:
# Load the prepared dataset
data_3 = pd.read_csv("data/burgers_data_with_derivatives.csv")
Xf_3 = data_3.drop(columns=['u_t'])  # Features
u_t_pred_3 = data_3['u_t']  # Target

array([[ 2.22044605e-016, -5.19795723e-009,  1.87407944e-007, ...,
         1.51776028e-200,  1.87221261e-227,  1.52161214e-054],
       [-4.31008024e-011,  3.81872445e-009,  1.01125870e-007, ...,
         2.28726169e-205,  2.39202073e-233,  1.09369989e-056],
       [ 4.77340778e-010,  7.44277652e-009,  8.61568350e-008, ...,
         1.99828316e-201,  1.10107265e-229,  3.03611273e-057],
       ...,
       [ 1.57232607e-004,  1.21973454e-004,  5.78700062e-005, ...,
         1.01106812e-105,  1.13395164e-122,  1.25784819e-034],
       [ 1.64970064e-004,  1.25618402e-004,  5.82651566e-005, ...,
         1.73357011e-105,  1.99791535e-122,  1.32822407e-034],
       [ 1.72934908e-004,  1.29256598e-004,  5.81571227e-005, ...,
         2.68703097e-105,  3.07386168e-122,  1.30864936e-034]])

In [45]:
# Train the symbolic regression model
# Generate variable names based on the feature columns
variable_names_3 = ['u',
 'u_x',
 'u_xx',
 'u@u',
 'u@u_x',
 'u@u_xx',
 'u_x@u_x',
 'u_x@u_xx',
 'u_xx@u_xx',
 'u@u@u',
 'u@u@u_x',
 'u@u@u_xx',
 'u@u_x@u_x',
 'u@u_x@u_xx',
 'u@u_xx@u_xx',
 'u_x@u_x@u_x',
 'u_x@u_x@u_xx',
 'u_x@u_xx@u_xx',
 'u_xx@u_xx@u_xx']

complexity_options = {
    "complexity_of_operators": {
        "+": 1,  # Discourage repeated usage of `+`
        "*": 3,  # Allow multiplication since it's integral to the structure
        "-": 1,  # Allow subtraction
        "/": 10,  # Strongly discourage division unless necessary
        "sin": 100,  # Exclude unnecessary functions
        "cos": 100
    },
    "complexity_of_constants": 5,  # Ensure constants have minimal impact on complexity
    "complexity_of_variables": 5  # Discourage variable repetition
}


model_5 = PySRRegressor(
    niterations=2,  # Number of iterations (adjust as necessary)
    
    # List of binary operators (two inputs)
    binary_operators=[
        "+",    # Addition
       
        '*'     # Multiplication
    ],
    extra_jax_mappings="""
        using SymbolicRegression
        # Ensure no variable-variable multiplication
        function is_variable_only(expr)
            return contains_variables(expr) && !contains_constants(expr)
        end

        # Set constraints:
        # * Constants can multiply variables
        # * Variables can only add
        constraints = Dict(
            "+" => is_variable_only,
            "*" => SymbolicRegression.contains_constants
        )
    """,
    constraints={
        "+": {"lhs": "variable", "rhs": "variable"},  # Variables add
        "*": {"lhs": "constant", "rhs": "variable"}  # Constants multiply variables
    },
    loss="L2DistLoss()",  # Default loss, or specify custom loss if needed

    
    
    # Define any custom operators (optional)
    # extra_sympy_mappings={"inv": lambda x: 1/x},  # Example of a custom operator
    variable_names = variable_names_3,
    model_selection='accuracy',                    # Enable model selection mode           
    select_k_features=2,
    complexity_of_operators=complexity_options["complexity_of_operators"],
    complexity_of_constants=complexity_options["complexity_of_constants"],
    complexity_of_variables=complexity_options["complexity_of_variables"],
    population_size=20,               # Population size for evolutionary search
    batching=True,
    parsimony=1e-5,
    batch_size=5000,                    # Batching for large datapoint
    maxsize=11                          # Maximum complexity of equations
)
    





c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:915: FutureWarning: `loss` has been renamed to `elementwise_loss` in PySRRegressor. Please use that instead.
  warnings.warn(
c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:922: FutureWarning: `variable_names` is a data-dependent parameter and should be passed when fit is called. Ignoring parameter; please pass `variable_names` during the call to fit instead.
  warnings.warn(


In [47]:
from pysr import PySRRegressor

# Define variable names and complexity options
variable_names = ['u',
 'u_x',
 'u_xx',
 'u@u',
 'u@u_x',
 'u@u_xx',
 'u_x@u_x',
 'u_x@u_xx',
 'u_xx@u_xx',
 'u@u@u',
 'u@u@u_x',
 'u@u@u_xx',
 'u@u_x@u_x',
 'u@u_x@u_xx',
 'u@u_xx@u_xx',
 'u_x@u_x@u_x',
 'u_x@u_x@u_xx',
 'u_x@u_xx@u_xx',
 'u_xx@u_xx@u_xx']

complexity_of_operators = {
    "+": 1,
    "*": 3,
    "-": 1,
    "/": 10,
    "sin": 100,
    "cos": 100,
}

# Symbolic regression model
model_5 = PySRRegressor(
    niterations=2,                  # Increase iterations for better results
    binary_operators=["+", "*"],       # Limited to addition and multiplication
    constraints={
        "+": {"lhs": "variable", "rhs": "variable"},
        "*": {"lhs": "constant", "rhs": "variable"},
    },
    loss="L2DistLoss()",               # Define the loss function
    variable_names=variable_names,     # Ensure this matches your dataset
    model_selection="accuracy",        # Prioritize accuracy
    population_size=20,                # Control population size
    batching=True,
    batch_size=5000,                   # Adjust based on dataset size
    parsimony=1e-5,                    # Regularization strength
    maxsize=11,                        # Max complexity of equations
    complexity_of_operators=complexity_of_operators,
    complexity_of_constants=5,
    complexity_of_variables=5,
)


c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:915: FutureWarning: `loss` has been renamed to `elementwise_loss` in PySRRegressor. Please use that instead.
  warnings.warn(
c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:922: FutureWarning: `variable_names` is a data-dependent parameter and should be passed when fit is called. Ignoring parameter; please pass `variable_names` during the call to fit instead.
  warnings.warn(


In [51]:
# Define variable names and complexity options
variable_names = ['u',
 'u_x',
 'u_xx',
 'u@u',
 'u@u_x',
 'u@u_xx',
 'u_x@u_x',
 'u_x@u_xx',
 'u_xx@u_xx',
 'u@u@u',
 'u@u@u_x',
 'u@u@u_xx',
 'u@u_x@u_x',
 'u@u_x@u_xx',
 'u@u_xx@u_xx',
 'u_x@u_x@u_x',
 'u_x@u_x@u_xx',
 'u_x@u_xx@u_xx',
 'u_xx@u_xx@u_xx']


# Symbolic regression model
model_5 = PySRRegressor(
    niterations=2,  # Increase iterations for meaningful results
    binary_operators=["+", "*"],  # Limited to addition and multiplication
    
    loss="L2DistLoss()",  # Loss function
    variable_names=variable_names,  # Match your dataset
    model_selection="accuracy",  # Prioritize accuracy
    population_size=20,  # Population size for evolutionary search
    batching=True,
    batch_size=5000,  # Adjust based on dataset size
    parsimony=1e-5,  # Regularization strength
    maxsize=11  # Max complexity of equations
)


c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:915: FutureWarning: `loss` has been renamed to `elementwise_loss` in PySRRegressor. Please use that instead.
  warnings.warn(
c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:922: FutureWarning: `variable_names` is a data-dependent parameter and should be passed when fit is called. Ignoring parameter; please pass `variable_names` during the call to fit instead.
  warnings.warn(


In [52]:
model_5.fit(Xf, u_t_pred)

c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2059: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, and consider using a larger `maxsize`.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 30 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 30 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 4.620e+02
Head worker occupation: 46.3%. This is high, and will prevent efficient resource usage. Increase `ncycles

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                   -0.81351906 * x1   
	1        0.224044                             (-1.1063926 + x3) * x1   
	2        0.001795             (-0.47034022 + (-0.6554943 + x3)) * x1   
	3  >>>>  0.003797  (((0.5319054 * x2) * -0.2432936) + x1) * (x3 +...   
	
	       loss  complexity  
	0  0.001900           3  
	1  0.001214           5  
	2  0.001210           7  
	3  0.001192          11  
]

In [73]:
from sklearn.preprocessing import StandardScaler

# Normalize X and rescale y
scaler = StandardScaler()
X_normalized = scaler.fit_transform(Xf)

In [76]:
X_normalized


array([[-1.71498585e+00, -1.72529815e+00, -6.10678921e-01,
        -4.91815463e-08,  7.61854115e-07],
       [-1.71498585e+00, -1.71176640e+00, -6.10678921e-01,
         3.61316503e-08,  4.11098690e-07],
       [-1.71498585e+00, -1.69823465e+00, -6.10678918e-01,
         7.04213677e-08,  3.50246302e-07],
       ...,
       [ 1.71498585e+00,  1.69823465e+00, -6.09812157e-01,
         1.15407703e-03,  2.35254181e-04],
       [ 1.71498585e+00,  1.71176640e+00, -6.09769504e-01,
         1.18856446e-03,  2.36860554e-04],
       [ 1.71498585e+00,  1.72529815e+00, -6.09725597e-01,
         1.22298801e-03,  2.36421373e-04]])

In [77]:
X_df = pd.DataFrame(X_normalized, columns=variable_names)

In [78]:
X_df

,t,x,u,u_x,u_xx
0,-1.714986,-1.725298,-0.610679,-4.918155e-08,7.618541e-07
1,-1.714986,-1.711766,-0.610679,3.613165e-08,4.110987e-07
2,-1.714986,-1.698235,-0.610679,7.042137e-08,3.502463e-07
3,-1.714986,-1.684703,-0.610679,1.380305e-07,-4.194012e-07
4,-1.714986,-1.671171,-0.610679,-5.159703e-08,-1.296055e-06
...,...,...,...,...,...
25851,1.714986,1.671171,-0.609894,1.085838e-03,2.361803e-04
25852,1.714986,1.684703,-0.609854,1.120121e-03,2.345498e-04
25853,1.714986,1.698235,-0.609812,1.154077e-03,2.352542e-04
25854,1.714986,1.711766,-0.609770,1.188564e-03,2.368606e-04


In [ ]:
model_5.fit(X_df, y_rescaled)


c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


In [ ]:
model_5.fit(Xf, u_t_pred)

In [ ]:
predictions_rescaled = model.predict(X_normalized)
predictions = (predictions_rescaled * (0.1 * 10**17)) + np.mean(y)

In [ ]:
model.save("pysr_results.pkl")

In [53]:
data = scipy.io.loadmat("C:/Users/chidi/Downloads/burgers.mat")
        
t = np.real(data['t'].flatten()[:,None])
x = np.real(data['x'].flatten()[:,None])
Exact = np.real(data['usol']).T
        
X, T = np.meshgrid(x,t)
        
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.flatten()[:,None] 

In [75]:
u_star

array([[2.22044605e-16],
       [4.85722573e-16],
       [1.04083409e-15],
       ...,
       [1.29915917e-04],
       [1.50017788e-04],
       [1.72934908e-04]])

In [74]:
Exact

array([[ 2.22044605e-16,  4.85722573e-16,  1.04083409e-15, ...,
        -7.80625564e-17, -1.82145965e-17,  2.08166817e-17],
       [-4.31008024e-11,  4.28848658e-11, -4.26225071e-11, ...,
         4.34146816e-11, -4.33821173e-11,  4.32728853e-11],
       [ 4.77340778e-10, -4.78508736e-10,  4.79589604e-10, ...,
        -4.73120376e-10,  4.74640424e-10, -4.76044357e-10],
       ...,
       [ 1.81398207e-04,  2.08914676e-04,  2.40188515e-04, ...,
         1.17517897e-04,  1.36050551e-04,  1.57232607e-04],
       [ 1.90083736e-04,  2.18644653e-04,  2.51064261e-04, ...,
         1.23619041e-04,  1.42928147e-04,  1.64970064e-04],
       [ 1.99013820e-04,  2.28635026e-04,  2.62217886e-04, ...,
         1.29915917e-04,  1.50017788e-04,  1.72934908e-04]])

In [69]:
X_star

array([[-8.    ,  0.    ],
       [-7.9375,  0.    ],
       [-7.875 ,  0.    ],
       ...,
       [ 7.8125, 10.    ],
       [ 7.875 , 10.    ],
       [ 7.9375, 10.    ]])

In [68]:
X.shape

(101, 256)

In [63]:
x.shape

(256, 1)

In [60]:

from pyDOE import lhs
# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0)    
    
# Measurement data
# In this case, measurements are from N_u_s points and continuously sampled all the time.
N_u_s = 10 
idx_s = np.random.choice(x.shape[0], N_u_s, replace=False)
X0 = X[:, idx_s]
T0 = T[:, idx_s]
Exact0 = Exact[:, idx_s]

N_u_t = int(t.shape[0]*1)
idx_t = np.random.choice(t.shape[0], N_u_t, replace=False)
X0 = X0[idx_t, :]
T0 = T0[idx_t, :]
Exact0 = Exact0[idx_t, :]

X_u_meas = np.hstack((X0.flatten()[:,None], T0.flatten()[:,None]))
u_meas = Exact0.flatten()[:,None]   

# Training measurements, which are randomly sampled spatio-temporally
Split_TrainVal = 0.8
N_u_train = int(X_u_meas.shape[0]*Split_TrainVal)
idx_train = np.random.choice(X_u_meas.shape[0], N_u_train, replace=False)
X_u_train = X_u_meas[idx_train,:]
u_train = u_meas[idx_train,:]

# Validation Measurements, which are the rest of measurements
idx_val = np.setdiff1d(np.arange(X_u_meas.shape[0]), idx_train, assume_unique=True)
X_u_val = X_u_meas[idx_val,:]
u_val = u_meas[idx_val,:]
        
# Collocation points
N_f = 50000

X_f_train = lb + (ub-lb)*lhs(2, N_f)
#    X_f_train = lb + (ub-lb)*sobol_seq.i4_sobol_generate(2, N_f)        
X_f_train = np.vstack((X_f_train, X_u_train))

# Option: Add noise
noise = 0.1
u_train = u_train + noise*np.std(u_train)*np.random.randn(u_train.shape[0], u_train.shape[1])
u_val = u_val + noise*np.std(u_val)*np.random.randn(u_val.shape[0], u_val.shape[1])


In [77]:
X0.shape

(101, 10)

In [73]:
u_train.shape

(808, 1)

In [71]:
X_u_train.shape

(808, 2)

In [67]:
X_f_train.shape

(50808, 2)

In [65]:
u_meas.shape

(1010, 1)

In [61]:
idx_s

array([147, 193, 123,  45,  15, 172,  41,   2,  68,  27], dtype=int32)

In [58]:
X_star.shape


(25856, 2)

In [56]:
u_star.shape

(25856, 1)

In [54]:
Exact

array([[ 2.22044605e-16,  4.85722573e-16,  1.04083409e-15, ...,
        -7.80625564e-17, -1.82145965e-17,  2.08166817e-17],
       [-4.31008024e-11,  4.28848658e-11, -4.26225071e-11, ...,
         4.34146816e-11, -4.33821173e-11,  4.32728853e-11],
       [ 4.77340778e-10, -4.78508736e-10,  4.79589604e-10, ...,
        -4.73120376e-10,  4.74640424e-10, -4.76044357e-10],
       ...,
       [ 1.81398207e-04,  2.08914676e-04,  2.40188515e-04, ...,
         1.17517897e-04,  1.36050551e-04,  1.57232607e-04],
       [ 1.90083736e-04,  2.18644653e-04,  2.51064261e-04, ...,
         1.23619041e-04,  1.42928147e-04,  1.64970064e-04],
       [ 1.99013820e-04,  2.28635026e-04,  2.62217886e-04, ...,
         1.29915917e-04,  1.50017788e-04,  1.72934908e-04]])